In [2]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from Functions import *

### Working on UpdateGoogle Sheet function
This will basically pull activities that are missing from the Google Sheet, saving time from doing a full pull.

In [48]:
# Importing credentials for Strava's API
from Credentials import StravaCredentials

# Copy and paste this link in your browser and extract code -> #https://www.strava.com/oauth/authorize?client_id=99205&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
data = StravaCredentials.data

# In case you need to get the code again uncomment these:
# webbrowser.open(f"https://www.strava.com/oauth/authorize?client_id={data['client_id']}&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all")
data['code'] = input("From the web broswer enter the code:")

In [49]:
# Creating function that returns the access token that is used in the other api calls
def GetToken(data):
    token = requests.post(url= 'https://www.strava.com/api/v3/oauth/token',data=data).json()
    access_token = token['access_token']
    
    return access_token

access_token = GetToken(data)

In [52]:
def retrieve_activities(access_token):
    url = "https://www.strava.com/api/v3/activities"
    activities = pd.DataFrame()
    page = 1
    while True:
        # get page of activities from Strava
        print('Getting page number:', page)
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        
        # check for rate limit exceeded error
        if r.status_code != 200:
            print('Error:',r.status_code, 'stopping extraction')
            break
    
        else:
            r = r.json()
            print(f'Extraction of page {page} complete')
            # if no results then exit loop
            if (not r):
                print('Extraction done')
                break
            r = pd.json_normalize(r)
            # adding the new table to the data frame that is storing all the data
            activities = pd.concat([activities, r])
            page += 1

    try:
        # clean up the dataframe
        clean_activities = activities[['id',
            'name',
            'distance',
            'elapsed_time',
            'total_elevation_gain',
            'sport_type',
            'start_date',
            'achievement_count',
            'athlete_count',
            'start_latlng',
            'end_latlng',
            'average_speed',
            'max_speed',
            'average_temp',
            'average_heartrate',
            'max_heartrate',
            'average_cadence',
            'elev_high',
            'elev_low']]
        return clean_activities
    except:
        ('Error occurred during extraction')

# call the function to retrieve activities
activities = retrieve_activities(access_token)
clean_activities = activities

Getting page number: 1
Extraction of page 1 complete
Getting page number: 2
Extraction of page 2 complete
Getting page number: 3
Extraction of page 3 complete
Getting page number: 4
Extraction of page 4 complete
Getting page number: 5
Extraction of page 5 complete
Getting page number: 6
Extraction of page 6 complete
Extraction done


In [56]:
id_list = clean_activities['id']
id_list.to_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Outputs\idlist.csv')

In [3]:
id_list = pd.read_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Outputs\idlist.csv')
id_list = list(id_list['id'])

In [23]:
# Setting up parameters for gspread - updating google sheet

service_file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'
myscope = ['https://spreadsheets.google.com/feeds', 
            'https://www.googleapis.com/auth/drive']

mycred = ServiceAccountCredentials.from_json_keyfile_name(service_file_path,myscope) # type: ignore

client = gspread.authorize(mycred)

mysheet = client.open('workout-data').sheet1

list_of_row = mysheet.get_all_values()

df = pd.DataFrame(list_of_row)

In [31]:
df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,activity_id,name,start_date,sport_type,distance,workout_time_min,,,calories,total_elevation_gain,...,total_elevation_gain_score,average_heartrate_score,max_heartrate_score,avg_lap_time_score,lap_count_score,avg_speed_score,max_speed_score,effort_score,effort_score_label,effort_score_rank
1,9984187254,Afternoon Ride,2023-10-05,Ride,12.67,47.19,3.72,,406,142,...,10,10,15,15,5,20,30,205,High Effort,4
2,9978422018,Evening Workout,2023-10-04,Functional-Cardio Workout,0,28.3,,,397,0,...,1,30,30,1,35,1,1,155,High Effort,4
3,9971958248,Evening Workout,2023-10-03,Functional-Cardio Workout,0,16.46,,,166,0,...,1,10,15,1,25,1,1,80,No Effort,1
4,9971957772,Afternoon Hike,2023-10-03,Hike,4.22,63.23,14.98,,278,68,...,5,5,15,20,5,5,15,150,High Effort,4
5,9965043232,Afternoon Workout,2023-10-02,Functional-Cardio Workout,0,36.59,,,430,0,...,1,15,25,5,35,1,1,154,High Effort,4
6,9957999970,Post run wod,2023-10-01,Functional-Cardio Workout,0,20.08,,,241,0,...,1,20,25,1,20,1,1,110,Low Effort,2
7,9957998721,Afternoon Run,2023-10-01,Run,5.01,28.06,5.60,,399,43,...,5,30,35,1,35,30,20,221,High Effort,4
8,9939802653,Afternoon Run,2023-09-28,Run,6,35.15,5.86,,477,42,...,5,30,30,5,35,30,20,235,High Effort,4
9,9933791721,Afternoon Workout,2023-09-27,Functional-Cardio Workout,0,24,#DIV/0!,,308,0,...,1,20,25,1,35,1,1,135,Medium Effort,3


In [46]:
f = open(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Outputs\all_workouts.json')
workout_json = json.load(f)